In [4]:
import xarray as xr
ds = xr.open_dataset(r"C:\Users\Ankit\ARGO_prototype\ARGO_datasets\1902673_prof.nc")

In [5]:
ds.data_vars

Data variables:
    DATA_TYPE                     object 8B ...
    FORMAT_VERSION                object 8B ...
    HANDBOOK_VERSION              object 8B ...
    REFERENCE_DATE_TIME           object 8B ...
    DATE_CREATION                 object 8B ...
    DATE_UPDATE                   object 8B ...
    PLATFORM_NUMBER               (N_PROF) object 552B ...
    PROJECT_NAME                  (N_PROF) object 552B ...
    PI_NAME                       (N_PROF) object 552B ...
    STATION_PARAMETERS            (N_PROF, N_PARAM) object 2kB ...
    CYCLE_NUMBER                  (N_PROF) float64 552B ...
    DIRECTION                     (N_PROF) object 552B ...
    DATA_CENTRE                   (N_PROF) object 552B ...
    DC_REFERENCE                  (N_PROF) object 552B ...
    DATA_STATE_INDICATOR          (N_PROF) object 552B ...
    DATA_MODE                     (N_PROF) object 552B ...
    PLATFORM_TYPE                 (N_PROF) object 552B ...
    FLOAT_SERIAL_NO               (N_P

In [6]:
vars_to_keep = [
    "JULD",          
    "LATITUDE",     
    "LONGITUDE",     
    "PRES",         
    "PRES_ADJUSTED", 
    "TEMP",          
    "TEMP_ADJUSTED", 
    "PSAL",          
    "PSAL_ADJUSTED"  
]

In [7]:
ds1 = ds[vars_to_keep]

In [8]:
df = ds1.to_dataframe().reset_index()

In [9]:
df = df[["JULD", "LATITUDE", "LONGITUDE", 
         "PRES_ADJUSTED", "TEMP_ADJUSTED", "PSAL_ADJUSTED"]]

In [10]:
df = df.rename(columns={
    "JULD": "time",
    "LATITUDE": "lat",
    "LONGITUDE": "lon",
    "PRES_ADJUSTED": "depth_dbar",
    "TEMP_ADJUSTED": "temperature_C",
    "PSAL_ADJUSTED": "salinity_PSU"
})

In [11]:
df = df.dropna().reset_index(drop=True)

In [14]:
df.to_csv(r"C:\Users\Ankit\ARGO_prototype\ARGO_datasets\argo_1902673_clean1.csv", index=False)

In [15]:
df.head(3)

,time,lat,lon,depth_dbar,temperature_C,salinity_PSU
0,2023-10-26 14:17:25,13.6,68.366667,0.1,29.847000,36.616001
1,2023-10-26 14:17:25,13.6,68.366667,0.8,29.847000,36.618000
2,2023-10-26 14:17:25,13.6,68.366667,2.0,29.853001,36.615002


In [16]:
import pandas as pd
import sqlite3
from sentence_transformers import SentenceTransformer
import numpy as np

In [17]:
csv_file = r"C:\Users\Ankit\ARGO_prototype\ARGO_datasets\argo_1902673_clean1.csv"
df = pd.read_csv(csv_file)

In [18]:
db_file = "argo_data.db"
conn = sqlite3.connect(db_file)

In [19]:
table_name = "argo_profiles"
df.to_sql(table_name, conn, if_exists="replace", index=False)

5817

In [20]:
query = f"""
SELECT time, lat, lon, depth_dbar, temperature_C, salinity_PSU
FROM {table_name}
WHERE depth_dbar < 200
LIMIT 5;
"""
sample = pd.read_sql(query, conn)
print(sample)

                  time   lat        lon  depth_dbar  temperature_C  \
0  2023-10-26 14:17:25  13.6  68.366667         0.1         29.847   
1  2023-10-26 14:17:25  13.6  68.366667         0.8         29.847   
2  2023-10-26 14:17:25  13.6  68.366667         2.0         29.853   
3  2023-10-26 14:17:25  13.6  68.366667         2.8         29.855   
4  2023-10-26 14:17:25  13.6  68.366667         3.7         29.854   

   salinity_PSU  
0        36.616  
1        36.618  
2        36.615  
3        36.614  
4        36.616  


In [21]:
def build_embeddings(df, text_cols=["time", "lat", "lon"]):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    texts = df[text_cols].astype(str).agg(" ".join, axis=1).tolist()
    embeddings = model.encode(texts, convert_to_numpy=True)
    return embeddings

In [22]:
import re
from transformers import pipeline
import tensorflow_hub as hub

In [23]:
nlp = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [24]:
def classify_intent(query: str) -> str:
    query_lower = query.lower()
    if "compare" in query_lower:
        return "compare_parameters"
    elif "nearest" in query_lower or "closest" in query_lower:
        return "find_nearest_floats"
    else:
        return "fetch_profile"

In [25]:
def extract_entities(query: str) -> dict:
    embedding = nlp([query])

    entities = {"variable": None, "time": None, "lat_range": None, "lon_range": None, "depth_range": None}

    q = query.lower()
    if "salinity" in q:
        entities["variable"] = "salinity_PSU"
    elif "temperature" in q or "temp" in q:
        entities["variable"] = "temperature_C"

    time_match = re.findall(r"(20\d{2})(?:[-/ ]?(0[1-9]|1[0-2]))?", q)
    if time_match:
        year, month = time_match[0]
        if month:
            entities["time"] = f"{year}-{month}"
        else:
            entities["time"] = year

    if "equator" in q:
        entities["lat_range"] = (-5, 5)
    if "indian ocean" in q:
        entities["lon_range"] = (20, 120)
        entities["lat_range"] = (-40, 25)

    depth_match = re.findall(r"(\d+)\s*(?:m|meter|dbar)", q)
    if depth_match:
        depth = int(depth_match[0])
        entities["depth_range"] = (depth - 5, depth + 5) 

    return entities, embedding

In [26]:
def parse_query(query: str):
    intent = classify_intent(query)
    entities, embedding = extract_entities(query)  # unpack both
    return {"intent": intent, "entities": entities, "embedding": embedding.numpy().tolist()}

In [27]:
if __name__ == "__main__":
    query = "Show me salinity profiles near the equator in March 2023"
    parsed = parse_query(query)
    print("User Query:", query)
    print("Parsed:", parsed)

User Query: Show me salinity profiles near the equator in March 2023
Parsed: {'intent': 'fetch_profile', 'entities': {'variable': 'salinity_PSU', 'time': '2023', 'lat_range': (-5, 5), 'lon_range': None, 'depth_range': None}, 'embedding': [[-0.02270379103720188, 0.0007957153720781207, -0.03364606946706772, 0.03108806163072586, 0.06267266720533371, -0.039996229112148285, -0.029263151809573174, 0.027397148311138153, 0.014960931614041328, 0.04412486031651497, -0.031215140596032143, -0.07838419824838638, -0.045297831296920776, 0.02091735228896141, 0.00471334857866168, -0.05087479203939438, -0.014090467244386673, 0.04242359474301338, 0.06769967824220657, 0.015359747223556042, -0.051088713109493256, 0.04822639375925064, 0.06875637173652649, 0.012384336441755295, 0.08588636666536331, -0.06049209460616112, -0.007108860183507204, 0.04837056249380112, -0.057659320533275604, 0.06590092182159424, -0.06062731891870499, 0.01908811926841736, 0.04085057973861694, -0.01757074147462845, -0.01020709145814

In [37]:
def build_sql_query(entities: dict) -> tuple[str, tuple]:
    """
    Build SQL query based on parsed entities.
    Returns (query, params).
    """
    base = "SELECT * FROM argo WHERE 1=1"
    conditions = []
    params = []

    if entities.get("variable"):
        var = entities["variable"]
        if var == "salinity_PSU":
            base = "SELECT profile_id, latitude, longitude, date, depth, salinity FROM argo WHERE 1=1"
        elif var == "temperature_C":
            base = "SELECT profile_id, latitude, longitude, date, depth, temperature FROM argo WHERE 1=1"

    if entities.get("time"):
        if len(entities["time"]) == 4:  # YYYY
            conditions.append("strftime('%Y', date) = ?")
            params.append(entities["time"])
        elif len(entities["time"]) == 7:  # YYYY-MM
            conditions.append("strftime('%Y-%m', date) = ?")
            params.append(entities["time"])

    if entities.get("lat_range"):
        conditions.append("latitude BETWEEN ? AND ?")
        params.extend(entities["lat_range"])

    if entities.get("lon_range"):
        conditions.append("longitude BETWEEN ? AND ?")
        params.extend(entities["lon_range"])

    if entities.get("depth_range"):
        conditions.append("depth BETWEEN ? AND ?")
        params.extend(entities["depth_range"])

    if conditions:
        base += " AND " + " AND ".join(conditions)

    return base, tuple(params)


In [38]:
def run_query(entities, db_file="argo_data.db", table_name="argo_profiles"):
    import sqlite3
    
    query = f"SELECT * FROM {table_name} WHERE 1=1"
    params = []

    # --- Apply filters ---
    if entities.get("variable"):
        query += f" AND {entities['variable']} IS NOT NULL"
    if entities.get("time"):
        query += " AND time LIKE ?"
        params.append(f"{entities['time']}%")
    if entities.get("lat_range"):
        query += " AND lat BETWEEN ? AND ?"
        params.extend(entities["lat_range"])
    if entities.get("lon_range"):
        query += " AND lon BETWEEN ? AND ?"
        params.extend(entities["lon_range"])
    if entities.get("depth_range"):
        query += " AND depth_dbar BETWEEN ? AND ?"
        params.extend(entities["depth_range"])

    conn = sqlite3.connect(db_file)
    cur = conn.cursor()
    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()

    return rows

In [39]:
if __name__ == "__main__":
    # Example from Step 3
    parsed = {
        "intent": "fetch_profile",
        "entities": {
            "variable": "salinity_PSU",
            "time": "2023-10",
            "lat_range": (13.6,13.7333),
            "lon_range": None,
            "depth_range": None
        }
    }

    results = run_query(parsed["entities"])
    print("SQL Results:")
    for r in results[:5]:  # show first 5 rows
        print(r)

SQL Results:
('2023-10-26 14:17:25', 13.6, 68.36666666666666, 0.1, 29.847, 36.616)
('2023-10-26 14:17:25', 13.6, 68.36666666666666, 0.8, 29.847, 36.618)
('2023-10-26 14:17:25', 13.6, 68.36666666666666, 2.0, 29.853, 36.615)
('2023-10-26 14:17:25', 13.6, 68.36666666666666, 2.8, 29.855, 36.614)
('2023-10-26 14:17:25', 13.6, 68.36666666666666, 3.7, 29.854, 36.616)


In [40]:
import matplotlib.pyplot as plt
import folium
import pandas as pd
import plotly.express as px

ModuleNotFoundError: No module named 'folium'